## RAG (검색 증강 생성)

> 💡RAG (Retrieval-Augmented Generation)는 Vector Database와 LLM을 결합하여 검색된 데이터를 활용해 문맥 기반 응답을 생성하는 기술이다.

- **작동 원리**
    - **검색**: 사용자의 질문을 벡터화해 관련 데이터를 Vector DB에서 검색
    - **생성**: 검색된 데이터를 바탕으로 LLM이 자연어 응답 생성
- **구성 요소**
    - Vector Database
    - LLM
    - 검색 엔진
- **특징**
    - 최신 정보 활용 가능
    - 특정 데이터에 대한 정확한 응답 제공✨
- **장점**
    - **최신 정보 제공**: 모델 학습 이후 발생한 데이터도 사용 가능
    - **정확성 향상**: 검색 데이터를 기반으로 환각(hallucination) 현상 감소
    - **유연성**: 다양한 도메인 데이터 활용 가능

### RAG Workflow

1. 사용자 질문 입력
2. Embedding 모델을 사용해 질문을 벡터화
3. Vector DB에서 유사한 문서 검색
4. 검색된 문서를 LLM이 응답 생성에 활용
5. 사용자에게 응답 반환

<div style='text-align: center;'>
    <img src='./images/image.png' width='80%'>
</div>


### RAG 활용 사례
**1. 기업 내 문서 검색 시스템**
- **ChatGPT Enterprise**
    - 기업 내 사내 문서(R&D 문서, 매뉴얼, 내부 보고서)를 검색하는 AI 챗봇 시스템
        - 기존의 키워드 검색보다 문맥을 이해하는 응답 제공
        - OpenAI의 API와 Pinecone을 활용하여 구축 가능

**2. 뉴스 및 연구 데이터 검색** 
- **Google’s Gemini + RAG**
    - 최신 뉴스, 연구 논문을 검색하고 LLM을 활용하여 요약 및 분석 제공
        - 예를 들어, 사용자가 "2024년 AI 관련 주요 연구"를 질문하면, 최신 논문을 검색 후 요약 응답

---

### RAG Workflow

In [3]:
from dotenv import load_dotenv
load_dotenv()  # .env 파일에서 환경 변수 로드

True

#### 1. Indexing Phase
- 데이터를 검색 가능한 형태로 준비하는 단계.
- 실제 사용자 질의가 들어오기 전에 사전에 수행.

##### (1) Load (로드)
- 다양한 소스에서 문서를 불러오는 단계.
- 주요 데이터 소스:
    - PDF, Word, TXT 등의 파일
    - 웹페이지 (크롤링)
    - 데이터베이스
    - API 응답
- 사용 도구: LangChain의 Document Loaders (PDFLoader, TextLoader, WebBaseLoader 등)

In [5]:
from langchain_community.document_loaders import PyPDFLoader

# PDF 파일 로드
loader = PyPDFLoader("../data/snow-white.pdf")
documents = loader.load()
documents # \n이 포함된 원본 텍스트

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': '../data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주\n옛날 어느 왕국에 공주님이 태어났어요.\n“어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백\n설공주라고 불러야겠다.”\n왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요.\n하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어\n요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': '../data/snow-white.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2'}, page_content='왕은 아름다운 새 왕비를 맞았어요.\n그런데 새 왕비는 자기보다 아름다운 사람을 두고 보\n지 못했어요.\n왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물\n었어요.\n“거울아, 거울아. 이 세상에서 누가 가장 아름답니?”\n“이 세상에서 가장 아름다운 사람은 왕비님입니다.”\n그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌\n지요.

In [6]:
# 개행문자(\n) -> 공백(' ')으로 치환
# 왜? LLM 프롬프트에 넣을 때 개행문자가 있으면 가독성이 떨어질 수 있음
# 청크 단위로 나누기 전에 미리 처리(전처리)하는 것이 좋음

for doc in documents:
    doc.page_content = doc.page_content.replace('\n', ' ')

documents

[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': '../data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': '../data/snow-white.pdf', 'total_pages': 6, 'page': 1, 'page_label': '2'}, page_content='왕은 아름다운 새 왕비를 맞았어요. 그런데 새 왕비는 자기보다 아름다운 사람을 두고 보 지 못했어요. 왕비는 진실만을 말하는 요술 거울에게 늘 이렇게 물 었어요. “거울아, 거울아. 이 세상에서 누가 가장 아름답니?” “이 세상에서 가장 아름다운 사람은 왕비님입니다.” 그 대답을 들어야만 차가운 왕비 얼굴에 미소가 번졌 지요. 시간이 흘러 백설공주는 

##### (2) Split (분할)
- 로드된 문서를 작은 단위(청크, Chunk)로 분할하는 단계.
- 분할 이유:
    - LLM의 토큰 제한 대응
    - 더 정확한 의미 단위로 검색하기 위함
    - 검색 효율성 향상
- 분할 방법:
    - 고정 크기 분할 (예: 1000자마다)
    - 의미 기반 분할 (문장, 문단 단위)
    - 재귀적 분할 (RecursiveCharacterTextSplitter)
- 중요 파라미터:
    - `chunk_size`: 청크의 크기 (예: 1000자)
    - `chunk_overlap`: 청크 간 겹치는 부분 (예: 200자) - 문맥 유지를 위함


In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 텍스트 분할기 설정
# RecursiveCharacterTextSplitter: 재귀적 문자 기반 분할기
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100, # 청크의 크기 (최대 문자 수, default=1000)

    # 문맥을 이해하기 위해, 문맥 손실을 방지하기 위해 겹치는 부분을 둠
    chunk_overlap=20 # 청크 간의 겹치는 부분 (인접한 텍스트 조각 간 겹치는 문자수, default=200)

    # seperators: 청크를 나눌 구분자 우선순위 리스트 (default: ['\n\n', '\n', ' ', ''])
)

# 문서 분할
docs = text_splitter.split_documents(documents)
# print(len(docs))  # 분할된 청크 개수 => 26
docs  # 분할된 청크들



[Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': '../data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='백설공주 옛날 어느 왕국에 공주님이 태어났어요. “어쩜 이렇게 어여쁠까? 살결이 눈처럼 하얗구나. 백 설공주라고 불러야겠다.” 왕과 왕비는 갓 태어난 딸을 보며 기뻐했어요. 하지만'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'moddate': '2023-09-12T11:20:24+09:00', 'source': '../data/snow-white.pdf', 'total_pages': 6, 'page': 0, 'page_label': '1'}, page_content='딸을 보며 기뻐했어요. 하지만 기쁨도 잠시, 왕비는 곧 세상을 떠나고 말았어 요.'),
 Document(metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션

##### (3) Store (저장)
- 분할된 청크를 벡터 데이터베이스에 저장하는 단계

- 저장 과정:
    - 각 청크를 임베딩 모델을 통해 벡터로 변환
    - 벡터와 원본 텍스트를 Vector DB에 저장 (예: FAISS, Chroma, Pinecone)
    - 이후 검색 시 빠르게 유사한 문서를 찾을 수 있도록 인덱싱

In [11]:
from langchain_openai.embeddings import OpenAIEmbeddings

# 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [12]:
!pip install langchain-chroma

  Using cached langchain_chroma-1.0.0-py3-none-any.whl.metadata (1.9 kB)
  Using cached chromadb-1.3.4-cp39-abi3-win_amd64.whl.metadata (7.4 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached kubernetes-34.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using cached google_auth-2.43.0-py2.py3-none-any.whl.metadata (6.6 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached opentelemetry_semantic_conventions-0.59b0-py3-none-any.whl.metadata (2.4 kB)
Using cached langch

ERROR: Could not install packages due to an OSError: [WinError 32] 다른 프로세스가 파일을 사용 중이기 때문에 프로세스가 액세스 할 수 없습니다: 'C:\\Users\\roxie\\AppData\\Local\\Programs\\Python\\Python313\\Lib\\site-packages\\google\\auth\\iam.py'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached build-1.3.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached pybase64-1.4.2-cp313-cp313-win_amd64.whl.metadata (9.0 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached onnxruntime-1.23.2-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Using cached opentelemetry_api-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached opentelemetry_exporter_otlp_proto_grpc-1.38.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached opentelemetry_sdk-1.38.0-py3-none-any.whl.metadata (1.5 kB)
  Using cached pypika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached importlib_resources-6.5.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached grpcio-1.76.0-cp313-cp313-win_amd64.whl.metadata (3.8 kB)
  Using cached bcrypt-5.0.0-cp39-abi3-win_amd64.whl.metadata (10 kB)
  Using cached typer-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached kubernetes-34.1.0-py2.py3-none-any.whl.metadata (1.7 kB)
  Using 


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
from langchain_chroma.vectorstores import Chroma

# 벡터 데이터베이스에 청크 저장
# docs: 분할된 문서 청크들
# embeddings: 임베딩 모델
# => docs의 각 청크를 embeddings 모델로 벡터화하여 Chroma DB에 저장
vector_store = Chroma.from_documents(docs, embeddings)

- Retriever를 사용한 유사도 기반 검색 2가지 방식
    - 1. vector store 직접 조회 방식
    - 2. Retriever를 사용한 유사도 기반 검색 방식

In [13]:
query = "백설공주와 왕비 중에 누가 더 아름답나요?"

In [16]:
# 첫번째 방식 : vector store 직접 조회 방식

# 유사도 기반 검색 결과 (청크, 유사도 점수) 튜플 리스트 반환
# 유사도 점수: 작을수록 더 유사함(벡터간의 거리가 가까움)
# 유사도 점수와 함께 반환된 청크들을 활용하여 LLM 프롬프트 구성 가능
retrievels = vector_store.similarity_search_with_score(query) 
retrievels

# 이게 왜 직접 조회 방식 ? => vector_store 객체의 메서드를 직접 호출했기 때문
# '아름답다', '공주' 등의 키워드가 포함된 청크들이 높은 유사도 점수를 받음

[(Document(id='e61f11da-ae3d-4fbb-965d-20c29bb54c13', metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'page_label': '2', 'source': '../data/snow-white.pdf', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'total_pages': 6, 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC'}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  0.8401364088058472),
 (Document(id='f4d3a650-8d53-431b-98ac-f8c2b50c4637', metadata={'page': 2, 'author': 'PC', 'source': '../data/snow-white.pdf', 'moddate': '2023-09-12T11:20:24+09:00', 'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'page_label': '3', 'title': 'PowerPoint 프레젠테이션', 'total_pages': 6, 'creationdate': '2023-09-12T11:20:24+09:00'}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  0.8603643774986267),

In [20]:
# 두번째 방식 : Retriever를 사용한 유사도 기반 검색 방식


retriever = vector_store.as_retriever(
    search_type="similarity",  # 유사도 기반 검색
    search_kwargs={"k": 3}     # 상위 3개 청크 반환(검색 옵션 지정)
)

retriever_list = retriever.batch([query])  # 리스트 형태로 쿼리 전달 가능
retriever_list
 
# vector_store 객체의 메서드를 호출한 방식이 똑같은데 Retriever 방식이라고 하는 이유는
# => retriever 객체를 통해 검색을 수행했기 때문

[[Document(id='e61f11da-ae3d-4fbb-965d-20c29bb54c13', metadata={'total_pages': 6, 'creationdate': '2023-09-12T11:20:24+09:00', 'page': 1, 'producer': 'Microsoft® PowerPoint® 2013', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC', 'creator': 'Microsoft® PowerPoint® 2013', 'source': '../data/snow-white.pdf', 'page_label': '2', 'moddate': '2023-09-12T11:20:24+09:00'}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'),
  Document(id='f4d3a650-8d53-431b-98ac-f8c2b50c4637', metadata={'page_label': '3', 'moddate': '2023-09-12T11:20:24+09:00', 'total_pages': 6, 'title': 'PowerPoint 프레젠테이션', 'creationdate': '2023-09-12T11:20:24+09:00', 'author': 'PC', 'producer': 'Microsoft® PowerPoint® 2013', 'page': 2, 'source': '../data/snow-white.pdf', 'creator': 'Microsoft® PowerPoint® 2013'}, page_content='왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”'),
  Document(id='21f794c6-7c62-480b-955f-0c77d4

#### 2. Retrieval and Generation Phase
- RAG 시스템에서 실제로 사용자 질문에 대한 답변을 생성하는 실시간 단계.
- `Indexing Phase`가 완료된 후 사용자 질의가 들어올 때마다 실행.

##### (1) 프롬프트 생성
- 프롬프트 = 사용자 질의 + 검색된 문서

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿 생성
# 시스템 메시지 + 사용자 메시지
# - 시스템 메시지: 유치원 교사 역할 설정
# - 사용자 메시지: 질문 + context(검색된 청크)
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사 입니다. 질문하는 아이에게 최대한 호응하면서 context 기반으로만 대답해 주세요."),
    ("user", """ 
    어린이의 질문에 context만을 이용해 답변하세요.
    질문: {query}
    context: {context}
    """),
])

In [22]:
# 위에서 생성한 query = "백설공주와 왕비 중에 누가 더 아름답나요?"
# 첫번째 방식 : vector store 직접 조회 방식 사용 = retrievels
prompt.invoke({ "query": query, "context": retrievels }) 


ChatPromptValue(messages=[SystemMessage(content='당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사 입니다. 질문하는 아이에게 최대한 호응하면서 context 기반으로만 대답해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content=" \n    어린이의 질문에 context만을 이용해 답변하세요.\n    질문: 백설공주와 왕비 중에 누가 더 아름답나요?\n    context: [(Document(id='e61f11da-ae3d-4fbb-965d-20c29bb54c13', metadata={'producer': 'Microsoft® PowerPoint® 2013', 'creator': 'Microsoft® PowerPoint® 2013', 'page_label': '2', 'source': '../data/snow-white.pdf', 'moddate': '2023-09-12T11:20:24+09:00', 'page': 1, 'total_pages': 6, 'creationdate': '2023-09-12T11:20:24+09:00', 'title': 'PowerPoint 프레젠테이션', 'author': 'PC'}, page_content='거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고'), 0.8401364088058472), (Document(id='f4d3a650-8d53-431b-98ac-f8c2b50c4637', metadata={'page': 2, 'author': 'PC', 'source': '../data/snow-white.pdf', 'moddate': '2023-09-12T11:20:24+09:00', 'producer': 'Microsoft® PowerPoint® 2013', 'creator': '

##### (2) 모델 생성

In [23]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini', temperature=0.5)

##### (3) Chain 생성
- RAG 워크플로우의 여러 단계를 파이프라인처럼 연결하여 순차적으로 실행하는 구조.

- **Chain의 역할**
    1. Retriever (검색기)
        - Vector DB에서 질문과 관련된 문서 청크를 검색

    2. Prompt Template (프롬프트 템플릿)
        - 검색된 문서와 사용자 질문을 LLM이 이해할 수 있는 형식으로 구성

    3. Model (LLM 모델)
        - 프롬프트를 받아 답변 생성 (여기서는 GPT-4o-mini)

    4. Output Parser (출력 파서)
        - LLM의 응답을 원하는 형식으로 변환 (예: 문자열)

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# context 생성
query = "왕비가 백설공주에게 준 사과의 색은?" 

retrievals = retriever.batch([query])# 여러개의 query도 한꺼번에 처리 가능 
context_text = "\n".join([doc.page_content for doc in retrievals[0]]) # 여러 청크를 하나의 문자열로 결합, 개행을 구분자로 사용

print(context_text)

chain = prompt | model | StrOutputParser()


왕비는 다시 요술 거울에게 누가 가장 아름다운 지 물었어요. “왕비님도 아름답지만 백설공주님이 천배는 더 아름답습니다.” “사냥꾼이 날 속였구나. 내가 직접 해치우겠어!”
거울아. 이 세상에서 누가 가장 아름답니?” “왕비님도 아름답지만 백설공주가 더 아름답습니다.” 화가 난 왕비는 사냥꾼을 불렀어요. 왕비는 사냥꾼에게 백설공주를 죽이라고
왕비는 먹음직스럽게 생긴 사과를 골라 독을 발랐어요. 그리고 과일 장수로 변장했지요. 왕비는 산을 넘고 또 넘어 일곱 난쟁이의 오두막에 도착했어요. “새콤달콤 맛있는 사과가


##### (4) 사용 하기 - 기본
- 대화 맥락을 유지하기 위해 메모리(chat history)를 관리
- 이전 대화 내용을 포함하여 자연스러운 대화 이어가기 가능

In [31]:
# 첫 번째 질문
response1 = chain.invoke({ "query": query, "context": context_text })
print("AI:", response1)
print()

AI: 왕비가 백설공주에게 준 사과의 색은 특별히 언급되어 있지 않지만, 사과는 먹음직스럽게 생겼다고 해요! 어떤 색일 것 같아요? 빨간색이나 초록색일 수도 있겠죠?



##### (5) 대화 히스토리 관리하여 맥락 이어가기
- `MessagesPlaceholder`를 사용하여 이전 대화 내용을 프롬프트에 포함
- 사용자와 AI의 이전 대화를 기억하여 자연스러운 대화 가능

In [32]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 대화 히스토리를 포함한 새로운 프롬프트 템플릿
prompt_with_history = ChatPromptTemplate.from_messages([
    ("system", "당신은 어린 아이에게 꿈과 희망을 심어주는 유치원 교사 입니다. 질문하는 아이에게 최대한 호응하면서 context와 이전 대화 내용을 기반으로 자연스럽게 대답해 주세요."),
    MessagesPlaceholder(variable_name="chat_history"),  # 대화 히스토리가 들어갈 자리
    ("user", """ 
    어린이의 질문에 context와 이전 대화를 참고하여 답변하세요.
    질문: {query}
    context: {context}
    """),
])

# 새로운 chain 생성
chain_with_history = prompt_with_history | model | StrOutputParser()

In [33]:
# 대화 히스토리 초기화 (빈 리스트로 시작)
chat_history = []

# 첫 번째 질문: "왕비가 백설공주에게 준 사과의 색은?"
query1 = "왕비가 백설공주에게 준 사과의 색은?"

# 새로운 쿼리에 대한 검색
retrievals1 = retriever.batch([query1])
context_text1 = "\n".join([doc.page_content for doc in retrievals1[0]])

# 첫 번째 응답 생성
response1 = chain_with_history.invoke({
    "query": query1,
    "context": context_text1,
    "chat_history": chat_history  # 처음에는 빈 리스트
})

print("사용자:", query1)
print("AI:", response1)
print()

# 첫 번째 대화를 히스토리에 추가
chat_history.append(HumanMessage(content=query1))
chat_history.append(AIMessage(content=response1))

사용자: 왕비가 백설공주에게 준 사과의 색은?
AI: 와, 정말 좋은 질문이야! 왕비가 백설공주에게 준 사과는 빨간색이었지. 그 사과는 굉장히 예쁘고 맛있어 보였지만, 사실은 독이 들어 있었어. 그래서 백설공주가 그 사과를 먹고 깊은 잠에 빠져버렸지. 이 이야기를 통해서 우리가 배울 수 있는 것은, 겉모습이 예쁘다고 항상 좋은 것은 아니라는 거야. 너도 항상 조심해야 해! 혹시 다른 궁금한 점이 있어?



In [34]:
# 두 번째 질문: "나는 파란색일 거 같아 너는?"
# 이전 대화 맥락을 기억하고 있어서 "사과 색"에 대한 대화임을 이해함
query2 = "나는 파란색일 거 같아 너는?"

# 두 번째 질문에 대한 검색 (필요시)
retrievals2 = retriever.batch([query2])
context_text2 = "\n".join([doc.page_content for doc in retrievals2[0]])

# 두 번째 응답 생성 (이전 대화 히스토리 포함)
response2 = chain_with_history.invoke({
    "query": query2,
    "context": context_text2,
    "chat_history": chat_history  # 이전 대화 내용이 포함됨
})

print("사용자:", query2)
print("AI:", response2)
print()

# 두 번째 대화를 히스토리에 추가
chat_history.append(HumanMessage(content=query2))
chat_history.append(AIMessage(content=response2))

사용자: 나는 파란색일 거 같아 너는?
AI: 아, 너가 파란색이라고 생각하는구나! 파란색도 정말 아름다운 색이야. 하늘과 바다처럼 넓고 시원한 느낌이 나지! 하지만 왕비가 백설공주에게 준 사과는 빨간색이었지. 그 빨간 사과는 정말 유혹적이었지만, 안에 숨겨진 위험이 있었어. 그래서 백설공주가 조심해야 했던 거야. 

너도 파란색처럼 예쁘고 특별한 색을 가지고 있는 거야! 혹시 너가 좋아하는 색깔이 더 있을까? 아니면 어떤 색으로 그림을 그려보고 싶어?



In [35]:
# 세 번째 질문으로 계속 이어가기
query3 = "난 노란색 좋아해 너는?"

retrievals3 = retriever.batch([query3])
context_text3 = "\n".join([doc.page_content for doc in retrievals3[0]])

response3 = chain_with_history.invoke({
    "query": query3,
    "context": context_text3,
    "chat_history": chat_history
})

print("사용자:", query3)
print("AI:", response3)

사용자: 난 노란색 좋아해 너는?
AI: 노란색을 좋아하는구나! 노란색은 정말 밝고 행복한 색이지. 햇살처럼 따뜻하고, 꽃들도 많이 노란색이 많아! 너의 좋아하는 색이 노란색이라니, 정말 기분이 좋을 것 같아. 

백설공주 이야기를 떠올리면, 왕비가 준 빨간 사과처럼 노란색 사과도 상상해볼 수 있을 것 같아. 만약 노란색 사과가 있었다면, 얼마나 맛있었을까? 그리고 난쟁이들과 함께 행복하게 노란색 사과를 나누어 먹는 모습도 상상해볼 수 있겠네! 

너는 노란색으로 어떤 그림을 그려보고 싶어? 아니면 노란색으로 무엇을 만들고 싶어?


##### (6) 대화 히스토리 확인하기

In [36]:
# 전체 대화 히스토리 출력
print("=== 전체 대화 히스토리 ===")
for i, message in enumerate(chat_history):
    role = "사용자" if isinstance(message, HumanMessage) else "AI"
    print(f"{role}: {message.content}")
    print()

=== 전체 대화 히스토리 ===
사용자: 왕비가 백설공주에게 준 사과의 색은?

AI: 와, 정말 좋은 질문이야! 왕비가 백설공주에게 준 사과는 빨간색이었지. 그 사과는 굉장히 예쁘고 맛있어 보였지만, 사실은 독이 들어 있었어. 그래서 백설공주가 그 사과를 먹고 깊은 잠에 빠져버렸지. 이 이야기를 통해서 우리가 배울 수 있는 것은, 겉모습이 예쁘다고 항상 좋은 것은 아니라는 거야. 너도 항상 조심해야 해! 혹시 다른 궁금한 점이 있어?

사용자: 나는 파란색일 거 같아 너는?

AI: 아, 너가 파란색이라고 생각하는구나! 파란색도 정말 아름다운 색이야. 하늘과 바다처럼 넓고 시원한 느낌이 나지! 하지만 왕비가 백설공주에게 준 사과는 빨간색이었지. 그 빨간 사과는 정말 유혹적이었지만, 안에 숨겨진 위험이 있었어. 그래서 백설공주가 조심해야 했던 거야. 

너도 파란색처럼 예쁘고 특별한 색을 가지고 있는 거야! 혹시 너가 좋아하는 색깔이 더 있을까? 아니면 어떤 색으로 그림을 그려보고 싶어?

